# Linear Regression Implementation

In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as la
from collections import Counter

In [2]:
from time_series import load_data, split_trips # this is available from one of my other python notebooks
vdf, _ = load_data('bus_train.db')
all_trips = split_trips(vdf)

In [8]:
def label_and_truncate(trip, bus_stop_coordinates):
    """ Given a dataframe of a trip following the specification in the previous homework assignment,
        generate the labels and throw away irrelevant rows. 
        
        Args: 
            trip (dataframe): a dataframe from the list outputted by split_trips from homework 2
            stop_coordinates ((float, float)): a pair of floats indicating the (latitude, longitude) 
                                               coordinates of the target bus stop. 
            
        Return:
            (dataframe): a labeled trip that is truncated at Forbes and Morewood and contains a new column 
                         called `eta` which contains the number of minutes until it reaches the bus stop. 
        """
    
    idx_array = pd.DataFrame()
    idx_array['dist'] = ((trip.lat - bus_stop_coordinates[0])**2 + (trip.lon - bus_stop_coordinates[1])**2)
    idx_array['idx'] = range(len(idx_array))
    
    min_value = idx_array['dist'].idxmin()
    idx_array.set_index(idx_array.idx, inplace = True)
    min_d = idx_array[idx_array['dist']==idx_array['dist'].min()].index.values
    
    eta_ = (min_value - trip.index).total_seconds() / 60.0
    trip['eta'] = eta_
    return trip.head(min_d[0]+1)

    
morewood_coordinates = (40.444671114203, -79.94356058465502) # (lat, lon)
labeled_trips = [label_and_truncate(trip, morewood_coordinates) for trip in all_trips]
labeled_vdf = pd.concat(labeled_trips).reset_index()
labeled_vdf = labeled_vdf[labeled_vdf["eta"] < 10*60].reset_index(drop=True)
print(Counter([len(t) for t in labeled_trips]))
print(labeled_vdf.head())


Counter({1: 506, 21: 200, 18: 190, 20: 184, 19: 163, 16: 162, 22: 159, 17: 151, 23: 139, 31: 132, 15: 128, 2: 125, 34: 112, 32: 111, 33: 101, 28: 98, 14: 97, 35: 95, 30: 95, 29: 93, 24: 90, 25: 89, 37: 86, 39: 83, 27: 83, 38: 82, 36: 77, 26: 75, 40: 70, 13: 62, 41: 53, 44: 52, 42: 47, 6: 44, 12: 39, 46: 39, 5: 39, 7: 38, 3: 36, 47: 33, 45: 33, 43: 31, 48: 27, 49: 26, 4: 26, 11: 25, 50: 25, 10: 23, 51: 23, 8: 19, 9: 18, 53: 16, 54: 15, 52: 14, 55: 14, 56: 8, 58: 3, 59: 3, 57: 3, 60: 3, 61: 1, 67: 1, 62: 1})
               tmstmp   vid        lat        lon  hdg   pid   rt        des  \
0 2016-08-11 10:56:00  5549  40.439504 -79.996981  114  4521  61A  Swissvale   
1 2016-08-11 10:57:00  5549  40.439504 -79.996981  114  4521  61A  Swissvale   
2 2016-08-11 10:58:00  5549  40.438842 -79.994733  124  4521  61A  Swissvale   
3 2016-08-11 10:59:00  5549  40.437938 -79.991213   94  4521  61A  Swissvale   
4 2016-08-11 10:59:00  5549  40.437938 -79.991213   94  4521  61A  Swissvale   

   pdis

## Generating Basic Features

In [29]:
import math
def create_features(vdf):
    """ Given a dataframe of labeled and truncated bus data, generate features for linear regression. 
    
        Args:
            df (dataframe) : dataframe of bus data with the eta column and truncated rows
        Return: 
            (dataframe) : dataframe of features for each example
        """
    vdf.reset_index()
    vdf["bias"] = 1
    vdf["day"] = vdf.tmstmp.dt.weekday
    vdf["hour"] = vdf.tmstmp.dt.hour
    vdf["min"] = (vdf.tmstmp.dt.hour * 60) + vdf.tmstmp.dt.minute
    
    vdf["sin_day_of_week"] = vdf["day"].apply(lambda x: math.sin((2*math.pi)*(x/7)))
    vdf["cos_day_of_week"] = vdf["day"].apply(lambda x: math.cos((2*math.pi)*(x/7)))
    vdf["sin_hour_of_day"] = vdf["hour"].apply(lambda x: math.sin((2*math.pi)*(x/24)))
    vdf["cos_hour_of_day"] = vdf["hour"].apply(lambda x: math.cos((2*math.pi)*(x/24)))
    vdf["sin_time_of_day"] = vdf["min"].apply(lambda x: math.sin((2*math.pi)*(x/1440)))
    vdf["cos_time_of_day"] = vdf["min"].apply(lambda x: math.cos((2*math.pi)*(x/1440)))
    vdf["sin_hdg"] = vdf["hdg"].apply(lambda x: math.sin((2*math.pi)*(x/360)))
    vdf["cos_hdg"] = vdf["hdg"].apply(lambda x: math.cos((2*math.pi)*(x/360)))
    
    vdf['weekday'] = np.where(vdf.day<=4, 1,0)
    
    rt = pd.get_dummies(vdf['rt'])
    dest = pd.get_dummies(vdf['des'])
    vdf = pd.merge(vdf, dest, left_index=True, right_index=True)
    vdf = pd.merge(vdf, rt, left_index=True, right_index=True)
    
    vdf.drop(['tmstmp','vid','pid','hdg','tablockid','tatripid','rt', 'des','day','hour','min'], axis = 1, inplace = True)
    
    return vdf

vdf_features = create_features(labeled_vdf)


In [30]:
with pd.option_context('display.max_columns', 26):
    print(vdf_features.columns)
    print(vdf_features.head())


Index(['lat', 'lon', 'pdist', 'spd', 'eta', 'bias', 'sin_hdg', 'cos_hdg',
       'sin_day_of_week', 'cos_day_of_week', 'sin_hour_of_day',
       'cos_hour_of_day', 'sin_time_of_day', 'cos_time_of_day', 'weekday',
       'Braddock ', 'Downtown', 'Greenfield Only', 'McKeesport ',
       'Murray-Waterfront', 'Swissvale', '61A', '61B', '61C', '61D'],
      dtype='object')
         lat        lon  pdist  spd   eta  bias   sin_hdg   cos_hdg  \
0  40.439504 -79.996981   1106    0  16.0     1  0.913545 -0.406737   
1  40.439504 -79.996981   1106    0  15.0     1  0.913545 -0.406737   
2  40.438842 -79.994733   1778    8  14.0     1  0.829038 -0.559193   
3  40.437938 -79.991213   2934    7  13.0     1  0.997564 -0.069756   
4  40.437938 -79.991213   2934    7  13.0     1  0.997564 -0.069756   

   sin_day_of_week  cos_day_of_week  sin_hour_of_day  cos_hour_of_day  \
0         0.433884        -0.900969              0.5        -0.866025   
1         0.433884        -0.900969              0.5    

## Linear Regression using Ordinary Least Squares

In [31]:
class LR_model():
    """ Perform linear regression and predict the output on unseen examples. 
        Attributes: 
            beta (array_like) : vector containing parameters for the features """
    
    def __init__(self, X, y):
        """ Initialize the linear regression model by computing the estimate of the weights parameter
            Args: 
                X (array-like) : feature matrix of training data where each row corresponds to an example
                y (array like) : vector of training data outputs 
            """
        lambda_ = math.pow(10,-4)
        X_T_X = X.T.dot(X)
        stabilizer = np.identity(X.shape[1]) * lambda_
        invert_X_T_X = X_T_X + stabilizer
        cholesky_fact = la.cho_factor(invert_X_T_X)
        X_T_Y = X.T.dot(y)
        self.beta = la.cho_solve(cholesky_fact, X_T_Y)
        pass
        
    def predict(self, X_p): 
        """ Predict the output of X_p using this linear model. 
            Args: 
                X_p (array_like) feature matrix of predictive data where each row corresponds to an example
            Return: 
                (array_like) vector of predicted outputs for the X_p
            """
        return np.array(X_p.values.dot(self.beta)).flatten()
        pass



In [3]:
# Calculate mean squared error on both the training and validation set
def compute_mse(LR, X, y, X_v, y_v):
    """ Given a linear regression model, calculate the mean squared error for the 
        training dataset, the validation dataset, and for a mean prediction
        Args:
            LR (LR_model) : Linear model
            X (array-like) : feature matrix of training data where each row corresponds to an example
            y (array like) : vector of training data outputs 
            X_v (array-like) : feature matrix of validation data where each row corresponds to an example
            y_v (array like) : vector of validation data outputs 
        Return: 
            (train_mse, train_mean_mse, 
             valid_mse, valid_mean_mse) : a 4-tuple of mean squared errors
                                             1. MSE of linear regression on the training set
                                             2. MSE of predicting the mean on the training set
                                             3. MSE of linear regression on the validation set
                                             4. MSE of predicting the mean on the validation set
                         
            
    """
    y_pred = LR.predict(X)
    y_v_pred = LR.predict(X_v)
    
    y_mean = LR.predict(X).mean()
    mse_ltrain = ((y_pred - y) ** 2).mean(axis=None)
    mse_lvalid = ((y_v_pred - y_v) ** 2).mean(axis=None)
    mse_ptrain = ((y - y_mean) ** 2).mean(axis=None)
    mse_pvalid = ((y_v - y_mean) ** 2).mean(axis=None)
    
    return(mse_ltrain, mse_ptrain, mse_lvalid, mse_pvalid)



## TrueTime Predictions

In [6]:
def compare_truetime(LR, labeled_vdf, pdf):
    """ Compute the mse of the truetime predictions and the linear regression mse on entries that have predictions.
        Args:
            LR (LR_model) : an already trained linear model
            labeled_vdf (pd.DataFrame): a dataframe of the truncated and labeled bus data (same as the input to create_features)
            pdf (pd.DataFrame): a dataframe of TrueTime predictions
        Return: 
            (tt_mse, lr_mse): a tuple of the TrueTime MSE, and the linear regression MSE
        """
    
    columns = list(labeled_vdf)
    pdf_filtered = pdf[pdf['prdtm']!='']
    
    m_vdf = labeled_vdf.merge(pdf_filtered, how='inner')
    vdf_features = create_features(m_vdf.filter(columns, axis=1))
    
    X_df = vdf_features.loc[:, vdf_features.columns != 'eta']
    y_pred = LR.predict(X_df)
    
    m_vdf['eta_prediction'] = y_pred
    m_vdf['eta_'] = (m_vdf['prdtm'] - m_vdf['tmstmp']).apply(lambda x: x  / np.timedelta64(1,'m')).astype('int64')

    mse_o = ((m_vdf['eta_'] - m_vdf['eta']) ** 2).mean(axis=None)
    mse_p = ((m_vdf['eta_prediction'] - m_vdf['eta']) ** 2).mean(axis=None)
    return(mse_o, mse_p)
    
    
compare_truetime(LR, labeled_vdf_valid, pdf_valid)
